In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

/Users/aryamantepal/anaconda3/envs/ML-env/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/aryamantepal/anaconda3/envs/ML-env/lib/python3.11/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <CFED5F8E-EC3F-36FD-AAA3-2C6C7F8D3DD9> /Users/aryamantepal/anaconda3/envs/ML-env/lib/python3.11/site-packages/torchvision/image.so
  Expected in:     <A54E39C4-6B62-3303-9BE6-7DB88EB078BF> /Users/aryamantepal/anaconda3/envs/ML-env/lib/python3.11/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [7]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
# Hyper-parameters 
input_size = 784
hidden_size = 500
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

In [3]:
# MNIST dataset 
train_dataset = torchvision.datasets.MNIST(root='../../data', 
                                           train=True, 
                                           transform=transforms.ToTensor(),  
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data', 
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)


In [4]:
class Net(nn.Module):
    def __init__(self, input_size,hidden_size, num_classes):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size,hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        
        return out

In [5]:
model = Net(input_size, hidden_size,num_classes)

In [6]:
# loss & optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

In [8]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        # Move tensors to the configured device
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step [100/600], Loss: 2.2806
Epoch [1/5], Step [200/600], Loss: 2.2630
Epoch [1/5], Step [300/600], Loss: 2.2372
Epoch [1/5], Step [400/600], Loss: 2.2249
Epoch [1/5], Step [500/600], Loss: 2.1899
Epoch [1/5], Step [600/600], Loss: 2.1955
Epoch [2/5], Step [100/600], Loss: 2.1469
Epoch [2/5], Step [200/600], Loss: 2.1506
Epoch [2/5], Step [300/600], Loss: 2.1087
Epoch [2/5], Step [400/600], Loss: 2.0858
Epoch [2/5], Step [500/600], Loss: 2.0499
Epoch [2/5], Step [600/600], Loss: 2.0528
Epoch [3/5], Step [100/600], Loss: 1.9808
Epoch [3/5], Step [200/600], Loss: 2.0057
Epoch [3/5], Step [300/600], Loss: 1.9133
Epoch [3/5], Step [400/600], Loss: 1.9397
Epoch [3/5], Step [500/600], Loss: 1.9002
Epoch [3/5], Step [600/600], Loss: 1.8010
Epoch [4/5], Step [100/600], Loss: 1.7401
Epoch [4/5], Step [200/600], Loss: 1.7988
Epoch [4/5], Step [300/600], Loss: 1.7751
Epoch [4/5], Step [400/600], Loss: 1.7046
Epoch [4/5], Step [500/600], Loss: 1.7305
Epoch [4/5], Step [600/600], Loss:

In [9]:
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))


Accuracy of the network on the 10000 test images: 78.63 %
